<img src='./img/LogoWekeo_Copernicus_RGB_0.png' alt='' align='centre' width='30%'></img>


## First look at "basic" altimetry data for Sentinel-3, SRAL ##

    Version: 3.0
    Date:    13/07/2020
    Author:  Ben Loveday (InnoFlair, Plymouth Marine Laboratory) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the European Commission Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty, under the MIT license associated with this code repository.

As part of Copernicus, EUMETSAT operates the Surface Topography Mission (STM) on Sentinel-3. The mission consists of a package of instruments including the Synthetic Aperture radar altimetry (SRAL), Microwave Radiometer (MWR) and DORIS positioning antenna. Here we will look at the waveforms provided by this package of instruments.

For this notebook we will use Level-2 altimetry data from Sentinel-3. The main point of this exercise is to show you how altimetry data 'looks', and especially to highlight that altimetry data are not (for now) imagery or swath data. Thus you won't have a native grid of pixels, but a 'track' of data. Waveforms are the fundamental signal that altimeters receive, and from which they derive all their information about the ocean surface. The shape of a waveform is determined by how a radar pulse interacts with the Earth's surface. Much more information on waveforms can be found <a href = "http://www.altimetry.info/radar-altimetry-tutorial/how-altimetry-works/from-radar-pulse-to-altimetry-measurements/">here</a>.

<div class="alert alert-block alert-warning">
    <b>Get the WEkEO User credentials</b>
<hr>
If you want to download the data to use this notebook, you will need WEkEO User credentials. If you do not have these, you can register <a href="https://www.wekeo.eu/web/guest/user-registration" target="_blank">here</a>.


As usual, we begin by importing required libraries

In [ ]:
# to interpret paths and folder for any OS
import os, sys
# math library
import numpy as np
# reading of NetCDF data
import xarray as xr
# plotting library
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import dates
# mapping library
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import json
from IPython.core.display import display, HTML
import glob
from zipfile import ZipFile
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'wekeo-hda'))
import hda_api_functions as hapi

import warnings
warnings.filterwarnings('ignore')

In the box below, we will set up a few parameters to help our figure plotting later on

In [ ]:
#we will look at the plot within the notebooks
%matplotlib inline
plt.rcParams["figure.figsize"] = (16,10)
plt.ioff()
matplotlib.rcParams.update({'font.size': 16})

land_resolution = '50m'
land_poly = cfeature.NaturalEarthFeature('physical', 'land', land_resolution,
                                        edgecolor='k',
                                        facecolor=cfeature.COLORS['land'])

Now we will start our script, proper. First we need to get some data.

WEkEO provides access to a huge number of datasets through its **'harmonised-data-access'** API. This allows us to query the full data catalogue and download data quickly and directly onto the Jupyter Lab. You can search for what data is available <a href="https://wekeo.eu/data?view=catalogue">here</a>

In order to use the HDA-API we need to provide some authentication credentials, which comes in the form of an API key and API token. In this notebook we have provided functions so you can retrieve the API key and token you need directly. You can find out more about this process in the notebook on HDA access (wekeo_harmonized_data_access_api.ipynb) that can be found in the **wekeo-hda** folder on your Jupyterlab.


We will also define a few other parameters including where to download the data to, and if we want the HDA-API functions to be verbose. **Lastly, we will tell the notebook where to find the query we will use to find the data.** These 'JSON' queries are what we use to ask WEkEO for data. They have a very specific form, but allow us quite fine grained control over what data to get. You can find the example one that we will use here: **JSON_templates/EO_EUM_DAT_SENTINEL-3_SR_2_WAT___.json**

In [ ]:
# set this key to true to download data.
download_data = True

 # This reduces the resolution of the plot to conserve memory - increasing the number gives a coarser plot
grid_factor = 3 

# your WEkEO API username and password (needs to be in '  ')
user_name = 'USERNAME'
password = 'PASSWORD'

# Generate an API key
api_key = hapi.generate_api_key(user_name, password)
display(HTML('Your API key is: <b>'+api_key+'</b>'))

In [ ]:
# where the data should be downloaded to:
download_dir_path = os.path.join(os.getcwd(),'products')
# where we can find our data query form:
JSON_query_dir = os.path.join(os.getcwd(),'JSON_templates')
# HDA-API loud and noisy?
verbose = False

# make the output directory if required
if not os.path.exists(download_dir_path):
    os.makedirs(download_dir_path)

Now we have set how we want the script to run, we are ready to get some data. We start this process by telling the script what kind of data we want. In this case, this is SRAL level 2 data, which has the following designation on WEkEO: **EO:EUM:DAT:SENTINEL-3:SR_2_WAT___**.

In [ ]:
# SLSTR LEVEL 2 Dataset ID
dataset_id = "EO:EUM:DAT:SENTINEL-3:SR_2_WAT___"

Here, we use this dataset_id to find the correct, locally stored JSON query file which describes the data we want. The query file is called: **JSON_templates/EO_EUM_DAT_SENTINEL-3_SR_2_WAT___.json**

You can edit this query if you want to get different data, but be aware of asking for too much data - you could be here a while and might run out of space to use this data in the JupyterLab. The box below gets the correct query file.

In [ ]:
# find query file
JSON_query_file = os.path.join(JSON_query_dir,dataset_id.replace(':','_')+".json")
if not os.path.exists(JSON_query_file):
    print('Query file ' + JSON_query_file + ' does not exist')
else:
    print('Found JSON query file for '+dataset_id)

Now we have a query, we need to launch it to WEkEO to get our data. The box below takes care of this through the following steps:
    1. initialise our HDA-API
    2. get an access token for our data
    3. accepts the WEkEO terms and conditions
    4. loads our JSON query into memory
    5. launches our search
    6. waits for our search results
    7. gets our result list
    8. downloads our data

This is quite a complex process, so much of the functionality has been buried 'behind the scenes'. If you want more information, you can check out the **wekeo-hda** tool kit in the parent training directory. The code below will report some information as it runs. At the end, it should tell you that one product has been downloaded.

In [ ]:
if download_data:
    HAPI_dict = hapi.init(dataset_id, api_key, download_dir_path)
    HAPI_dict = hapi.get_access_token(HAPI_dict)
    HAPI_dict = hapi.acceptTandC(HAPI_dict)

    # load the query
    with open(JSON_query_file, 'r') as f:
        query = json.load(f)

    # launch job
    print('Launching job...')
    HAPI_dict = hapi.get_job_id(HAPI_dict, query)

    # check results
    print('Getting results...')
    HAPI_dict = hapi.get_results_list(HAPI_dict)
    HAPI_dict = hapi.get_order_ids(HAPI_dict)

    # download data
    print('Downloading data...')
    HAPI_dict = hapi.download_data(HAPI_dict, file_extension='.zip')

In [ ]:
if download_data:
    # unzip file
    for filename in HAPI_dict['filenames']:
        if os.path.splitext(filename)[-1] == '.zip':
            print('Unzipping file')
            try:
                with ZipFile(filename, 'r') as zipObj:
                    # Extract all the contents of zip file in current directory
                    zipObj.extractall(os.path.dirname(filename))

                # clear up the zip file
                os.remove(filename)
            except:
                print("Failed to unzip....")

In [ ]:
if download_data:
    unzipped_file = HAPI_dict['filenames'][0].replace('.zip','.SEN3')
else:
    unzipped_file = glob.glob(os.path.join(download_dir_path,'*SR_2_WAT*.SEN3'))

Sentinel-3 SRAL level 2 data are provided in three versions (all in the same zipped folder): reduced, standard and enhanced. The nomenclature of the folder name for these products is SATELLITE_SR_level_WAT____begindate_time_enddate_time_XXX_cycle_orbit; so in the case of a Sentinel-3A level-2 altimetry product, a product would beging with the following designation:

S3A_SR_2_WAT____

Try the below cell on standard, reduced and enhanced file in turn. Note the differences between them (e.g. in the number of variables). The use of either of these files depends on your need and/or expertise and storage capability.

In [ ]:
#look at what is in a Sentinel-3A SRAL file folder
#data are in a folder parallel to the current one, named data
#to change from one file to the other, comment the current one, un-comment the one to test. 
input_root    = os.path.dirname(unzipped_file)
input_path    = os.path.basename(unzipped_file)
input_file = 'standard_measurement.nc'
#input_file = 'reduced_measurement.nc'
#input_file = 'enhanced_measurement.nc'

my_file = os.path.join(input_root,input_path,input_file)
nc = xr.open_dataset(my_file)

#list the variables within the file.
for variable in nc.variables:
    print(variable)

Now we will explore the size and statistics of the file. 

In [ ]:
# choose a variable at 1 Hz for now
# e.g. we choose the range in Ku band at 1 Hz
range_ku = nc.variables['range_ocean_01_ku'][:]
swh_ocean_01_ku = nc.variables['swh_ocean_01_ku'][:]
wind_speed_alt_01_ku = nc.variables['wind_speed_alt_01_ku'][:]
ssha_01_ku = nc.variables['ssha_01_ku'][:]

#flags
range_ocean_qual_01_ku = nc.variables['range_ocean_qual_01_ku'][:]
swh_ocean_qual_01_ku = nc.variables['swh_ocean_qual_01_ku'][:]
sig0_ocean_qual_01_ku = nc.variables['sig0_ocean_qual_01_ku'][:]
surf_class_01 = nc.variables['surf_class_01'][:]
rad_surf_type_01 = nc.variables['rad_surf_type_01'][:]

lat = nc.variables['lat_01'][:]
lon = nc.variables['lon_01'][:]

#show number of points, min max
print(['array length: ', np.shape(range_ku)])
print(['minimum: ', np.min(range_ku)])
print(['maximum: ', np.max(range_ku)])
print(['mean: ', np.mean(range_ku)])
print(['mean in km: ', np.mean(range_ku)/1000])
print(['standard deviation: ', np.std(range_ku)])

#try the same on the 20 Hz(at least for the shape of the array)
range_20_ku = nc.variables['range_ocean_20_ku'][:]
#show number of points, min max
print(['array length 20Hz: ', np.shape(range_20_ku)])

nc.close()

All Sentinel-3 level 2 marine products have quality flags associated with the geophysical variables. These give us confidence in our data, or describe why we cannot make retrievals of the parameters we are interested in. The next cell loads some data from these quality flags for range, significant wave height and surface roughness, as well as surface class information.

In [ ]:
# QC variables
range_ku[range_ocean_qual_01_ku != 0.0] = np.nan
swh_ocean_01_ku[swh_ocean_qual_01_ku != 0.0] = np.nan
ssha_01_ku[range_ocean_qual_01_ku != 0.0] = np.nan
ssha_01_ku[surf_class_01 != 0.0] = np.nan
ssha_01_ku[rad_surf_type_01 != 0.0] = np.nan
wind_speed_alt_01_ku[sig0_ocean_qual_01_ku != 0.0] = np.nan

Now lets plot our along track data (against latitude) and show where our flags suggest bad data.

In [ ]:
norm_range = range_ku - np.nanmin(range_ku)
norm_range = norm_range / np.nanmax(abs(norm_range))

flags_range1 = range_ocean_qual_01_ku.astype(float)
flags_range1[flags_range1 != 0.0] = 1.1
flags_range1[flags_range1 == 0.0] = np.nan

flags_range2 = swh_ocean_qual_01_ku.astype(float)
flags_range2[flags_range2 != 0.0] = 1.2
flags_range2[flags_range2 == 0.0] = np.nan

flags_range3 = surf_class_01.astype(float)
flags_range3[flags_range3 != 0.0] = 1.3
flags_range3[flags_range3 == 0.0] = np.nan

flags_range4 = sig0_ocean_qual_01_ku.astype(float)
flags_range4[flags_range4 != 0.0] = 1.4
flags_range4[flags_range4 == 0.0] = np.nan

fig1 = plt.figure(figsize=(10, 10), dpi=600)
ax = plt.subplot(1,1,1)
p1, = ax.plot(norm_range, lat, 'k', label='Normalised Range')
p2, = ax.plot(flags_range1, lat, 'ro',linewidth=0.0, label='Range quality flag is bad')
p3, = ax.plot(flags_range2, lat, 'bo',linewidth=0.0, label='Significant wave height quality flag is bad')
p4, = ax.plot(flags_range3, lat, 'go',linewidth=0.0, label='Surface class is not ocean')
p5, = ax.plot(flags_range4, lat, 'mo',linewidth=0.0, label='Sigma 0 quality flag is bad')
plt.xlabel('Range and flag values', fontsize=16)
plt.ylabel('Latitude', fontsize=16)

handles, labels = ax.get_legend_handles_labels()
plt.legend(handles, labels, fontsize=8)
plt.savefig('Quality_flags')
plt.show()

In the plot we can clearly see the flags raised where the track is likely over land/ice rather than ocean.

Now we will plot some  variables on a map. This will clearly show the data to be a'line' or 'track' of measurements along the orbit, and also that a file contains data from part of an orbit.

In [ ]:
fig1 = plt.figure(figsize=(20, 20), dpi=300)

#define a projection for our map
m = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))
#vmin and vmax for the scale are truncated slightly to help viewing; the satellite is at a distance around 315 km from the surface
low_pc = np.percentile(range_ku[np.isfinite(range_ku)],5)
high_pc = np.percentile(range_ku[np.isfinite(range_ku)],95)
range_ku[range_ku < low_pc] = low_pc
range_ku[range_ku > high_pc] = high_pc

f1 = m.scatter(lon, lat, c=range_ku, cmap='Spectral_r', marker='o', edgecolors=None, linewidth=0.0, vmin=low_pc, vmax=high_pc)
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07)
cbar.ax.tick_params(labelsize=16)
cbar.set_label('Range (1 Hz Ku) [m]', size=18)
plt.savefig('fig_range')
plt.show()

In [ ]:
fig1 = plt.figure(figsize=(20, 20), dpi=300)

#define a projection for our map
m = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))
#vmin and vmax for the scale are truncated slightly to help viewing; the satellite is at a distance around 315 km from the surface
low_pc = np.percentile(swh_ocean_01_ku[np.isfinite(swh_ocean_01_ku)],5)
high_pc = np.percentile(swh_ocean_01_ku[np.isfinite(swh_ocean_01_ku)],95)
swh_ocean_01_ku[swh_ocean_01_ku < low_pc] = low_pc
swh_ocean_01_ku[swh_ocean_01_ku > high_pc] = high_pc

f1 = m.scatter(lon, lat, c=swh_ocean_01_ku, cmap='Spectral_r', marker='o', edgecolors=None, linewidth=0.0, vmin=low_pc, vmax=high_pc)
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07)
cbar.ax.tick_params(labelsize=16)
cbar.set_label('Significant Wave Height (1 Hz Ku) [m]', size=18)
plt.savefig('fig_SWH')
plt.show()

In [ ]:
fig1 = plt.figure(figsize=(20, 20), dpi=300)

#define a projection for our map
m = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))
#vmin and vmax for the scale are truncated slightly to help viewing; the satellite is at a distance around 315 km from the surface
low_pc = np.percentile(wind_speed_alt_01_ku[np.isfinite(wind_speed_alt_01_ku)],5)
high_pc = np.percentile(wind_speed_alt_01_ku[np.isfinite(wind_speed_alt_01_ku)],95)
wind_speed_alt_01_ku[wind_speed_alt_01_ku < low_pc] = low_pc
wind_speed_alt_01_ku[wind_speed_alt_01_ku > high_pc] = high_pc

f1 = m.scatter(lon, lat, c=wind_speed_alt_01_ku, cmap='Spectral_r', marker='o', edgecolors=None, linewidth=0.0, vmin=low_pc, vmax=high_pc)
m.coastlines(resolution='50m', color='black', linewidth=1)
m.add_feature(cfeature.LAND, facecolor='0.75')
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07)
cbar.ax.tick_params(labelsize=16)
cbar.set_label('Wind Speed (1 Hz ku) [m.s$^{-1}$]', size=18)
plt.savefig('fig_WS')
plt.show()

In [ ]:
fig1 = plt.figure(figsize=(20, 20), dpi=300)

#define a projection for our map
m = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))
#vmin and vmax for the scale are truncated slightly to help viewing; the satellite is at a distance around 315 km from the surface
abs_ssha_01_ku = abs(ssha_01_ku)
high_pc = np.percentile(abs_ssha_01_ku[np.isfinite(ssha_01_ku)],95)
ssha_01_ku[ssha_01_ku < high_pc*-1] = high_pc*-1
ssha_01_ku[ssha_01_ku > high_pc] = high_pc

f1 = m.scatter(lon, lat, c=ssha_01_ku, cmap='RdBu_r', marker='o', edgecolors=None, linewidth=0.0, vmin=-1*high_pc, vmax=high_pc)
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07)
cbar.ax.tick_params(labelsize=16)
cbar.set_label('Sea Surface Height Anomaly Speed (1 Hz ku) [m]', size=18)
plt.savefig('fig_SSHA')
plt.show()

<img src='./img/all_partners_wekeo.png' alt='' align='center' width='75%'></img>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE">MIT License</a> <span style="float:right;"><a href="https://github.com/wekeo/wekeo-jupyter-lab">View on GitHub</a> | <a href="https://www.wekeo.eu/">WEkEO Website</a> | <a href=mailto:support@wekeo.eu>Contact</a></span></p>